In [0]:
from google.colab import files
files.upload()

{}

In [0]:
!python -m spacy download en_core_web_sm

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [0]:
import pandas as pd
df = pd.read_csv('Zones of India2.csv')
statename = list(set( df['State'].values.tolist()))
districts = list(set( df['District'].values.tolist()))
placenames = districts + statename
placenames = list(map(lambda x: x.lower(),placenames))


In [0]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [0]:
from joblib import load
tweets = []

f = open('PredictedTweets.txt','r')

for line in f.readlines():
    tweets.append(line)

commonWords = load('commonwords.pkl')

In [0]:
import re
def removePunctuations(sentence):
  punctList = '''!()-[]{};:'"\,./?@#$%^&@*_~'''
  withoutPunctuation=""
  for character in sentence:
    if(character not in punctList):
      withoutPunctuation+=character
  return withoutPunctuation

def cleanSentence(sentence):
  s = sentence.lower()
  s = [re.sub(r'http\S+','',s)]
  s = [re.sub(r'[^A-Za-z0-9 ]','', s[0])]
  s = [re.sub(' +', ' ', s[0])]
  s = removePunctuations(s[0])
  s = [word for word in s.split(' ') if(word not in commonWords)]
  s = ' '.join(s)
  return s

In [0]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer


def tokenizeData(data):
    return word_tokenize(data)


def tokenize_and_stem_data(newData):
  tokenized_data = []

  for sentence in newData:
    tokens = word_tokenize(sentence)
    tokenized_data.append(tokens)

  data = tokenized_data
  stemmer = SnowballStemmer('english')
  stemmedData = []
  for sentence in data:
    wordArray = []
    for word in sentence:
      # print(word)
      word = stemmer.stem(word)
      wordArray.append(word)
    stemmedData.append(' '.join(wordArray))

  return stemmedData

In [0]:
def numberReplaceWithToken(listOfTokens):
    tags = nltk.pos_tag(listOfTokens[:])
    #print(tags)
    for i in range(len(listOfTokens)):
        if(tags[i][1]=='CD'):
            listOfTokens[i] = "covidCount"
    return listOfTokens

In [0]:
tokenized_tweets = []
for tweet in tweets:
  cleanedSample =  cleanSentence(tweet)
  cleanedSample = [cleanedSample]
  # print(cleanedSample)
  tokens = word_tokenize(cleanedSample[0])
  tokenized_tweets.append(tokens)
  #print(tokens)

In [0]:
numberReplacedTweets = []
for tweet_tokens in tokenized_tweets:
    number_replaced_tweet = numberReplaceWithToken(tweet_tokens[:])
    numberReplacedTweets.append(number_replaced_tweet)



In [0]:
token_dict = {}
for tokenize_tweet in numberReplacedTweets:
  for token in tokenize_tweet:
    if(token not in token_dict.keys()):
      token_dict[token] = 1
    else:
      token_dict[token] += 1

In [0]:
token_list = token_dict.items()
token_list = list(token_list)
token_list

[('covidCount', 2455),
 ('more', 225),
 ('people', 163),
 ('including', 131),
 ('who', 34),
 ('returned', 12),
 ('kuwait', 1),
 ('may', 27),
 ('test', 144),
 ('positive', 537),
 ('covid19', 878),
 ('kerala', 29),
 ('active', 173),
 ('cases', 1045),
 ('health', 435),
 ('minister', 52),
 ('k', 12),
 ('shailaja', 8),
 ('death', 183),
 ('toll', 166),
 ('haryana', 34),
 ('rises', 131),
 ('fatality', 3),
 ('faridabad', 6),
 ('total', 428),
 ('after', 105),
 ('jhajjar', 3),
 ('sonipat', 5),
 ('amp', 158),
 ('gurgaon', 4),
 ('department', 238),
 ('gujarat', 60),
 ('reports', 50),
 ('new', 286),
 ('deaths', 202),
 ('taking', 165),
 ('case', 61),
 ('count', 17),
 ('fatalities', 35),
 ('official', 128),
 ('punjab', 28),
 ('noida', 5),
 ('gautam', 12),
 ('buddh', 9),
 ('nagar', 14),
 ('district', 71),
 ('up', 27),
 ('rise', 102),
 ('jammu', 30),
 ('kashmir', 31),
 ('number', 325),
 ('officials', 80),
 ('patients', 100),
 ('cured', 17),
 ('till', 61),
 ('now', 116),
 ('recovery', 12),
 ('rate', 16)

In [0]:
def createNGramCount(dataSetOfListOfTokens,n=3):
    ngramDict={}
    for listoftokens in dataSetOfListOfTokens:
        for i in range(len(listoftokens)-n):
            nGram = listoftokens[i:i+n]
            nGram = " ".join(nGram)
            if(nGram not in ngramDict.keys()):
                ngramDict[nGram] = 1
            else:
                ngramDict[nGram] += 1
    return ngramDict

In [0]:
trigram = createNGramCount(numberReplacedTweets,3)
trigram_list = trigram.items()
trigram_list = list(trigram_list)
trigram_list.sort(key=lambda x:x[1],reverse=True)
trigram_list = trigram_list[:20]
trigram_list = [x[0] for x in trigram_list]
print(trigram_list)



['covidCount new covid19', 'taking total number', 'total number cases', 'covid19 positive cases', 'total number positive', 'number positive cases', 'positive cases reported', 'new covid19 cases', 'new covid19 positive', 'covidCount including covidCount', 'cases rise covidCount', 'last covidCount hours', 'covidCount active cases', 'covidCount more people', 'active cases covidCount', 'people test positive', 'covidCount more covid19', 'today taking total', 'today total number', 'covidCount deaths reported']


In [0]:
bigram = createNGramCount(numberReplacedTweets,2)
bigram_list = bigram.items()
bigram_list = list(bigram_list)
bigram_list.sort(key=lambda x:x[1],reverse=True)
#print(bigram_list)
bigram_list=bigram_list[:20]
bigram_list = [x[0] for x in bigram_list]

#bigram_list = [bigram_list[0][0],bigram_list[2][0],bigram_list[-3][0]]
print(bigram_list)


['positive cases', 'covid19 cases', 'covidCount new', 'total number', 'cases covidCount', 'covidCount more', 'new covid19', 'covidCount deaths', 'cases reported', 'active cases', 'covid19 positive', 'taking total', 'death toll', 'rises covidCount', 'number cases', 'including covidCount', 'test positive', 'positive covid19', 'rise covidCount', 'covidCount health']


In [0]:
quadragram = createNGramCount(numberReplacedTweets,4)
quadragram_list = quadragram.items()
quadragram_list = list(quadragram_list)
quadragram_list.sort(key=lambda x:x[1],reverse=True)
quadragram_list=quadragram_list[:20]
quadragram_list=[x[0] for x in quadragram_list]
print(quadragram_list)



['total number positive cases', 'covidCount new covid19 cases', 'covidCount new covid19 positive', 'new covid19 positive cases', 'covid19 positive cases reported', 'taking total number cases', 'today taking total number', 'covid19 cases rise covidCount', 'taking total number positive', 'total number cases covidCount', 'covidCount including covidCount active', 'including covidCount active cases', 'total number cases state', 'covidCount more people test', 'number positive cases covidCount', 'death toll due covid19', 'number positive cases state', 'after covidCount people test', 'toll due covid19 rises', 'due covid19 rises covidCount']


In [0]:
countPatterns = []
for trigram in trigram_list:
  if('covidCount' in trigram):
    countPatterns.append(trigram)
for quadragram in quadragram_list:
  if('covidCount' in quadragram):
    countPatterns.append(quadragram)
countPatterns

['covidCount new covid19',
 'covidCount including covidCount',
 'cases rise covidCount',
 'last covidCount hours',
 'covidCount active cases',
 'covidCount more people',
 'active cases covidCount',
 'covidCount more covid19',
 'covidCount deaths reported',
 'covidCount new covid19 cases',
 'covidCount new covid19 positive',
 'covid19 cases rise covidCount',
 'total number cases covidCount',
 'covidCount including covidCount active',
 'including covidCount active cases',
 'covidCount more people test',
 'number positive cases covidCount',
 'after covidCount people test',
 'due covid19 rises covidCount']

In [0]:
joined = " ".join(countPatterns)
frequentWords = {}
for word in joined.split(" "):
  if(word in frequentWords):
    frequentWords[word] +=1
  else:
    frequentWords[word] =1
frequentWords

{'active': 4,
 'after': 1,
 'cases': 8,
 'covid19': 6,
 'covidCount': 21,
 'deaths': 1,
 'due': 1,
 'hours': 1,
 'including': 3,
 'last': 1,
 'more': 3,
 'new': 3,
 'number': 2,
 'people': 3,
 'positive': 2,
 'reported': 1,
 'rise': 2,
 'rises': 1,
 'test': 2,
 'total': 1}

In [0]:
requiredPatterns = []
for gram in countPatterns:
  if("new" in gram):
    requiredPatterns.append(gram)
  if("more" in gram):
    requiredPatterns.append(gram)
  if("positive" in gram):
    requiredPatterns.append(gram)
requiredPatterns = list(set(requiredPatterns))
requiredPatterns


['covidCount more covid19',
 'covidCount more people test',
 'covidCount more people',
 'covidCount new covid19 positive',
 'number positive cases covidCount',
 'covidCount new covid19',
 'covidCount new covid19 cases']

In [0]:
rulesList = []
for sentence in requiredPatterns:
  rule = []
  for word in sentence.split(" "):
    if(word == 'covidCount'):
      rule.append({"LIKE_NUM":True})
    else:
      rule.append({"LOWER":word})
  rulesList.append(rule)
rulesList

[[{'LIKE_NUM': True}, {'LOWER': 'more'}, {'LOWER': 'covid19'}],
 [{'LIKE_NUM': True},
  {'LOWER': 'more'},
  {'LOWER': 'people'},
  {'LOWER': 'test'}],
 [{'LIKE_NUM': True}, {'LOWER': 'more'}, {'LOWER': 'people'}],
 [{'LIKE_NUM': True},
  {'LOWER': 'new'},
  {'LOWER': 'covid19'},
  {'LOWER': 'positive'}],
 [{'LOWER': 'number'},
  {'LOWER': 'positive'},
  {'LOWER': 'cases'},
  {'LIKE_NUM': True}],
 [{'LIKE_NUM': True}, {'LOWER': 'new'}, {'LOWER': 'covid19'}],
 [{'LIKE_NUM': True},
  {'LOWER': 'new'},
  {'LOWER': 'covid19'},
  {'LOWER': 'cases'}]]

In [0]:
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy.matcher import Matcher

In [0]:
#tweet ="No new deaths, 425 new COVID19 positive cases reported in the last 24 hours. The total number of positive cases in Himachal pradesh is now 8895 (including 5254 active cases and 3518 recovered/discharged/migrated): Delhi Health Department"
#tweet = "3 persons tested #COVID19 positive in Kamrup Metro. One person is from Sarusajai Quarantine camp, 1 indoor patient of GMCH, & one has migrated to West Bengal. Total positive cases stand at 89: Assam Minister Himanta Biswa Sarma (15.5)"
#tweet = "40 new #COVID19 positive cases have been reported in Telangana today; taking the total number of positive cases to 1454. Total active cases stand at 461: Telangana Health Department"
tweet = "141 new #COVID19 positive cases and 5 deaths have been reported in Pune district today. Total positive cases stand at 3567 in the district and death toll is at 186: Dr Bhagawan Pawar, District Health Officer (DHO) #Maharashtra"
tweet = cleanSentence(tweet)
tweet = tweet.lower()
placeMatcherPattern = [{"TEXT":{"IN": placenames}}]
pattern = [{"LIKE_NUM": True},{"LOWER":"new"},{"LOWER":"covid19"},{"LOWER":"positive"}]
matcher = Matcher(nlp.vocab)
matcher.add('PLACE', None, placeMatcherPattern)
matcher.add('COUNT',None,[{'LIKE_NUM': True}, {'LOWER': 'more'}, {'LOWER': 'covid19'}],
 [{'LIKE_NUM': True},
  {'LOWER': 'more'},
  {'LOWER': 'people'},
  {'LOWER': 'test'}],
 [{'LIKE_NUM': True}, {'LOWER': 'more'}, {'LOWER': 'people'}],
 [{'LIKE_NUM': True},
  {'LOWER': 'new'},
  {'LOWER': 'covid19'},
  {'LOWER': 'positive'}],
 [{'LOWER': 'number'},
  {'LOWER': 'positive'},
  {'LOWER': 'cases'},
  {'LIKE_NUM': True}],
 [{'LIKE_NUM': True}, {'LOWER': 'new'}, {'LOWER': 'covid19'}],
 [{'LIKE_NUM': True},
  {'LOWER': 'new'},
  {'LOWER': 'covid19'},
  {'LOWER': 'cases'}])



COUNT 141 new covid19
COUNT 141 new covid19 positive
PLACE pune
PLACE maharashtra


In [0]:
doc = nlp(tweet)
matches = matcher(doc)
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]  # get the unicode ID, i.e. 'COLOR'
    span = doc[start : end]  # get the matched slice of the doc
    print(rule_id, span.text)
    

COUNT 141 new covid19
COUNT 141 new covid19 positive
PLACE pune
PLACE maharashtra


In [0]:
from joblib import dump
dump(placenames,'placenames.pkl')
files.download('placenames.pkl')

747

In [0]:
def createNGram(sentence,n=3):
    trigramsinTweet = [' '.join(sentence[i:i + n]) for i in range(0, len(sentence)-n)]
    return trigramsinTweet

In [0]:
i=0
for (index,tweet) in enumerate(numberReplacedTweets):
    trigramsinTweet = createNGram(tweet,3)
    bigramsinTweet = createNGram(tweet,2)
    quadgramsinTweet = createNGram(tweet,4)
    nGrams = bigramsinTweet+trigramsinTweet+quadgramsinTweet
    setofGrams =  set(nGrams)
    matchedGrams = list(setToCheck.intersection(setofGrams))
    if(len(matchedGrams) > 0):
        print(matchedGrams)
        bigramsinTokenizedTweets = createNGram(tokenized_tweets[index],2)
        trigramsinTokenizedTweets = createNGram(tokenized_tweets[index],3)
        quadgramsinTokenizedTweets = createNGram(tokenized_tweets[index],4)
        for gram in matchedGrams:
            if(len(gram.split(" ")) ==2):
                indexOfbigrams = bigramsinTweet.index(gram)
                count = bigramsinTokenizedTweets[indexOfbigrams]
                print(count)
            if(len(gram.split(" ")) ==3):
                indexOftrigrams = trigramsinTweet.index(gram)
                count = trigramsinTokenizedTweets[indexOftrigrams]
                print(count)
            if(len(gram.split(" ")) ==4):
                indexofQuadgrams = quadgramsinTweet.index(gram)
                count = quadgramsinTokenizedTweets[indexofQuadgrams]
                print(count)
                
        print(trigramsinTweet)       
        print(tweets[index])
        i+=1
print(i)

In [0]:
graphNodes = list(createNGramCount(numberReplacedTweets).keys())

graph = {}
for node in graphNodes:
  graph[node] = []

In [0]:
for tweet in numberReplacedTweets:
  trigrams = createNGram(tweet)
  for (index,trigram) in enumerate(trigrams[:-1]):
    fromnode = trigram
    #print(fromnode)
    tonode = trigrams[index+1]
    graph[fromnode].append(tonode)
    

In [0]:
checkTweet = "No new deaths, 425 new COVID19 positive cases reported in the last 24 hours. The total number of positive cases in Delhi is now 8895 (including 5254 active cases and 3518 recovered/discharged/migrated): Delhi Health Department"
cleanedSample =  cleanSentence(checkTweet)
cleanedSample = [cleanedSample]
tokenize_and_stem_sample = tokenize_and_stem_data(cleanedSample)
tokens = word_tokenize(tokenize_and_stem_sample[0])
numbertokens = numberReplaceWithToken(tokens)
checkTrigrams = createNGram(numbertokens)

for (index,trigram) in enumerate(checkTrigrams[:-1]):
    fromnode = trigram
    tonode = checkTrigrams[index+1]
    try:
      connectedFromNodes = graph[fromnode]
      print(fromnode)
      if(not(tonode in connectedFromNodes)):
        print("No sequence")
    except:
      print("Exception")

Exception
death covidCount new
covidCount new covid19
new covid19 posit
covid19 posit case
posit case report
case report last
report last covidCount
last covidCount hour
covidCount hour total
hour total number
total number posit
number posit case
posit case delhi
No sequence
Exception
Exception
now covidCount includ
covidCount includ covidCount
includ covidCount activ
covidCount activ case
activ case covidCount
No sequence
Exception
Exception


3

In [0]:
import numpy as np

def tweetToPOS(tweet):
    print(tweet)
    try:
        cleanedSample =  cleanSentence(tweet)
        cleanedSample = [cleanedSample]
        tokenize_and_stem_sample = tokenize_and_stem_data(cleanedSample)
        print(tokenize_and_stem_sample)
        tagged = nltk.pos_tag(tokenize_and_stem_sample[0]) 
        taggedtoPos = list(map(list, tagged))
        taggedtoPos = np.array(taggedtoPos)
        listOfPosWrods = taggedtoPos[:,1].tolist()
        print(listOfPosWrods)
        POSWords = ' '.join(listOfPosWrods)
        return POSWords
    except:
            print(tweet)
            return None
    
POSWordsArray=[]
for tweet in tweets:
    poswords = tweetToPOS(tweet)
    if(poswords != None):
        POSWordsArray.append(poswords)


In [0]:
def countExtractor(tweet):
    tags = nltk.pos_tag(tokenizeData(tweet))
    for tag in enumerate(tags):
        pos = tag[0]
        word,pos = tag[1]
        if pos == 'CD':
            

In [0]:
POSWordsArray = list(map(lambda x: x.split(" ")))

In [0]:
createNGram(POSWordsArray)

In [0]:
len(tweets)

886